In [1]:
import sys
import pandas as pd
import numpy as np

# Define a random seed for reproducibility
seed = 0
np.random.seed(seed)

import sklearn
import xgboost
import keras
import tensorflow as tf
import ast
import os
from joblib import dump, load


print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pd.__version__))
print('Numpy: {}'.format(np.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('XGBoost: {}'.format(xgboost.__version__))
print('Keras: {}'.format(keras.__version__))

C:\Users\brend\.conda\envs\NSQIP\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")
Using TensorFlow backend.


Python: 3.6.10 |Anaconda, Inc.| (default, May  7 2020, 19:46:08) [MSC v.1916 64 bit (AMD64)]
Pandas: 1.0.3
Numpy: 1.18.1
Sklearn: 0.22.1
XGBoost: 1.1.1
Keras: 2.3.1


In [2]:
def load_train_and_test(path):

    """Loads training features, training labels, testing features, and testing features
    Parameters:
        path (str) -- a single directory path containing all four datasets
    """

    train_features = pd.read_csv(path + 'train_features.csv', index_col=0)
    train_labels = pd.read_csv(path + 'train_labels.csv', index_col=0)
    test_features = pd.read_csv(path + 'test_features.csv', index_col=0)
    test_labels = pd.read_csv(path + 'test_labels.csv', index_col=0)

    return train_features, train_labels.values.ravel(), test_features, test_labels.values.ravel()

X_train, Y_train, X_test, Y_test = load_train_and_test('../data/split/')

In [3]:
# load a list of all optimization results files
dirName = '../reports/optimization/'

fileList = list()
dirList = list()

for (dirpath, dirnames, filenames) in os. walk(dirName):
    for file in filenames:
        if '.csv' in file:
            fileList.append(os.path. join(dirpath, file))

In [4]:
def load_model(name, hyperparameters):
    
    """ Loads the appropriate sklearn model from a model name and hyperparameters """
    
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    from sklearn.svm import SVC
    from xgboost import XGBClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.linear_model import LogisticRegression
    
    if name == 'AdaBoost':
        model = AdaBoostClassifier(**hyperparameters)
    elif name == 'DecisionTree':
        model = DecisionTreeClassifier(**hyperparameters)
    elif name == 'KMeans':
        model = KNeighborsClassifier(**hyperparameters)
    elif name == 'MLP':
        model = MLPClassifier(**hyperparameters)
    elif name == 'RandomForest':
        model = RandomForestClassifier(**hyperparameters)
    elif name == 'SVC':
        model = SVC(**hyperparameters, probability=True)
    elif name == 'XGBoost':
        model = XGBClassifier(**hyperparameters)
    elif name == 'GradientBoosting':
        model = GradientBoostingClassifier(**hyperparameters)
    elif name == 'LogisticRegression':
        model = LogisticRegression(**hyperparameters)
    else:
        print('Unkown model name')
        
    return model

In [5]:
import pickle

for file in fileList:
    #load hyperparameter optimization files
    results = pd.read_csv(file)
    
    # find the model_name from the filename
    file_split = file.split('/')[3]
    model_name = file_split.split('\\')[0]
    
    print(file)

    new_results = results.copy()

    # String to dictionary
    new_results['hyperparameters'] = new_results['hyperparameters'].map(ast.literal_eval)

    # Sort with best values on top
    new_results = new_results.sort_values('score', ascending = False).reset_index(drop = True)

    # Use best hyperparameters to create a model
    hyperparameters = new_results.loc[0, 'hyperparameters']
    
    # load the appropriate model and fit on training data
    model = load_model(model_name, hyperparameters)
    model.fit(X_train, Y_train)
    
    # create output filename
    new_dir = '../models/'
    filename = new_dir + model_name + '.sav'
    
    # save the trained model
    pickle.dump(model, open(filename, 'wb'))

../reports/optimization/AdaBoost\2021-01-17_bayes_test.csv
../reports/optimization/DecisionTree\2021-01-17_bayes_test.csv
../reports/optimization/GradientBoosting\2021-01-17_bayes_test.csv
../reports/optimization/LogisticRegression\2021-01-17_bayes_test.csv


C:\Users\brend\.conda\envs\NSQIP\lib\site-packages\sklearn\linear_model\_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


../reports/optimization/MLP\2021-01-17_bayes_test.csv
../reports/optimization/RandomForest\2021-01-17_bayes_test.csv
../reports/optimization/SVC\2021-01-17_bayes_test.csv
../reports/optimization/XGBoost\2021-01-17_bayes_test.csv
